## GCP

In [ ]:
# %pip list | grep google-cloud-aiplatform
# %pip list | grep google-api-core
# %pip install google-cloud-aiplatform==1.43.0
# %pip install google-api-core==2.17.1

In [ ]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, ChatSession, Part
import vertexai.preview.generative_models as generative_models

import geopandas as gpd
from shapely.geometry import Point

In [ ]:
vertexai.init(project = "adsp-capstone-property-pilot", location = "us-central1")

## Model Setting - Gemini pro 1.0

In [ ]:
model = GenerativeModel("gemini-1.0-pro")
chat = model.start_chat()

def get_chat_response(chat: ChatSession, prompt: str) -> str:
    text_response = []
    responses = chat.send_message(prompt, stream=True)
    for chunk in responses:
        text_response.append(chunk.text)
    return "".join(text_response)

In [ ]:
# Test chat
prompt = "Hello."
print(get_chat_response(chat, prompt))

## Settings

In [ ]:
import requests

url = "https://zillow-com1.p.rapidapi.com/propertyExtendedSearch"

headers = {
	"X-RapidAPI-Key": "735ee1fb05msh1468bdaed52d0a8p1c49cejsne0c7c5aea57a",
	"X-RapidAPI-Host": "zillow-com1.p.rapidapi.com"
}

## Import functions

In [ ]:
# import os
# os.chdir('/home/jupyter/property_pilot/scripts')

# import generate_api_filter
# import fetch_property_info
# import chat_response

In [ ]:
%run /home/jupyter/property_pilot/scripts/chat_response.py

In [ ]:
%run /home/jupyter/property_pilot/scripts/generate_api_filter.py

In [ ]:
%run /home/jupyter/property_pilot/scripts/fetch_property_info.py

## Import prompts

In [ ]:
os.chdir('/home/jupyter/property_pilot/prompts')

# prompt_classifier
file_path = 'instruction_classifier.txt'
try:
    with open(file_path, 'r') as file:
        instruction_prompt_classifier = file.read()
except FileNotFoundError:
    print("The file does not exist")
except Exception as e:
    print(f"An error occurred: {e}") 
    
# prompt_apifilter
file_path = 'generate_prompt_apifilter.txt'
try:
    with open(file_path, 'r') as file:
        instruction_prompt_apifilter = file.read()
except FileNotFoundError:
    print("The file does not exist")
except Exception as e:
    print(f"An error occurred: {e}")
    
# prompt_property(for final output)
file_path = 'generate_prompt_property.txt'
try:
    with open(file_path, 'r') as file:
        instruction_prompt_property = file.read()
except FileNotFoundError:
    print("The file does not exist")
except Exception as e:
    print(f"An error occurred: {e}")


## Property Plot main function

In [ ]:
def pp_chatbot(user_query):
    
    # User's quety classify
    prompt_classifier = generate_prompt_classifier(instruction_prompt_classifier, user_query)
    response_int = int(get_chat_response(chat, prompt_classifier))
    
    # if the user is looking for a property     
    #if response_int == 1:

    # get prompt for api filter 
    prompt_apifilter = generate_prompt_apifilter(instruction_prompt_apifilter, user_query)

    # get api filter
    apifilter = get_chat_response(chat, prompt_apifilter)
    apifilter = extract_json_to_dict(apifilter)

    # get listings from zillow
    response = requests.get(url, headers = headers, params = apifilter)

    # get top 3 listings(sorted by newest)
    # Define fields to extract
    fields = ["propertyType", "address", "price", "bedrooms", "bathrooms", "detailUrl", "imgSrc", "longitude", "latitude"]
    top_properties = extract_properties(response, 3, fields)

    # get description of the properties
    top_properties = fetch_descriptions(top_properties)

    # get resoFacts(detail) of the properties
    keys_to_fetch_resoFacts = [
        'hasGarage', 'hasPetsAllowed', 'heating', 'cooling', 'flooring', 'appliances',
        'laundryFeatures', 'associationFee',
        'livingArea', 'taxAnnualAmount', 'parkingFeatures', 'stories'
    ]
    top_properties = fetch_resoFacts(top_properties, headers, keys_to_fetch_resoFacts)

    # get school info of the properties
    top_properties = fetch_schools(top_properties, headers)

    # neighborhood boundaries shapefile
    neighborhood_boundaries_path = '/home/jupyter/Kshitiz_Working/geo_export_825d7df4-a9cd-4cef-b3d7-2ec1adc30204.shp'
    
    # neighborhood description file
    neighborhood_info_path = '/home/jupyter/Kshitiz_Working/neighborhood_info_final.csv'

    # load the neighborhood boundaries and description file
    neighborhoods = load_neighborhood_boundaries(neighborhood_boundaries_path)
    neighborhoods_info = load_neighborhood_info(neighborhood_info_path)

    # get neighborhood details of the properties
    top_properties = fetch_neighborhood(top_properties, neighborhoods)
    top_properties = fetch_neighborhood_info(top_properties, neighborhoods_info)

    # create property map
    property_map = create_property_map(top_properties)

    # change format
    fields_desc = fields + ['description', 'resoFacts', 'schools', 'neighborhood', 'neighborhood_description']
    property_info = format_properties(top_properties, fields_desc)

    # prompt for final output from Gemini
    prompt_final = generate_prompt_property(instruction_prompt_property, user_query, property_info, str(fields))

    # get final response from Gemini
    print(get_chat_response(chat, prompt_final))
    display(property_map)
    
    return top_properties

    #else:
        # print('The user is not looking for a property')
        # General reaponse from Gemini for now
        #print(get_chat_response(chat, user_query))

## Use case 1
### property search - detail info

In [ ]:
user_query = "I am looking for a two-bedroom apartment in Chicago."
top_properties = pp_chatbot(user_query)

In [ ]:
user_query = "Could you tell me about the first property in detail?"
print(get_chat_response(chat, user_query))

In [ ]:
user_query = "Which neighborhood is the first property located in? Tell me more about that neighborhood."
print(get_chat_response(chat, user_query))

In [ ]:
user_query = "What are some good schools nearby the first property?"
print(get_chat_response(chat, user_query))

In [ ]:
user_query = "I am interested in the first property. Should I buy it or rent it ?"
print(get_chat_response(chat, user_query))

In [ ]:
top_properties

## Use case 2
### Initial query: Neighborhood info

RAG

## Use case 3
### Initial query: Restaurant info

Yelp API